In [1]:
from requests.auth import HTTPBasicAuth
from requests import post
from requests import get
import json
import pandas as pd
import numpy as np
from time import sleep
import warnings
from fuzzywuzzy import fuzz
warnings.filterwarnings("ignore")
%matplotlib inline

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

import os
import glob

def ReadJsonFiles(filename):
    with open(filename) as infile:
        d = json.load(infile)
        return flatten_json(d)

C:\Users\mdejesus\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
WaveCustomer = pd.read_excel('C:/Users/mdejesus/TeleSignResponses/Wave/WaveCustomerRecord.xlsx')

In [3]:
WaveBatch = pd.read_csv('C:/Users/mdejesus/TeleSignResponses/Wave/WaveBatchTeleSign.csv')

In [4]:
WaveVerified = pd.merge(WaveCustomer, WaveBatch, how='left', left_on='CustomerKey', right_on='CustomerKey' )

##Contact information fields:
contact_firstname, contact_lastname, contact_zip_postal_code
'FirstName', 'LastName', 'Address1', 'ZIPCode'

In [5]:
WaveVerified['ContactName'] = WaveVerified.contact_firstname + ' ' + WaveVerified.contact_lastname
WaveVerified['CustomerName'] = WaveVerified.FirstName + ' ' + WaveVerified.LastName

In [12]:
WaveVerified['ContactNotAvail'] = np.where(WaveVerified.ContactName.isna(),1,0)

In [13]:
def get_ratio(row):
    if row['ContactNotAvail'] == 1:
        return 0
    else:
        return fuzz.token_sort_ratio(row['ContactName'], row['CustomerName'])


In [14]:
WaveVerified['NameMatch'] = WaveVerified.apply(get_ratio, axis=1)

In [3]:
WaveVerified['NameMatch'].hist()

NameError: name 'WaveVerified' is not defined

In [16]:
sum(WaveVerified.NameMatch >= 50) / sum(WaveVerified.NameMatch >= 0)

0.796

In [1]:
WaveVerified[WaveVerified.NameMatch > 0][['ContactName','CustomerName','NameMatch']].sort_values('NameMatch')

NameError: name 'WaveVerified' is not defined

In [19]:
path = 'C:/Users/mdejesus/TeleSignResponses/'

#path = 'C:/Users/mdejesus/TeleSignResponses/Wave'

MPPhoneScores = pd.DataFrame(ReadJsonFiles(filename) for filename in glob.glob(os.path.join(path,'resp_*.txt')))
   
MPPhoneID = pd.DataFrame(ReadJsonFiles(filename) for filename in glob.glob(os.path.join(path,'resppid_*.txt')))


In [20]:
droplist = ['blocklisting_block_code',
 'blocklisting_block_description',
 'blocklisting_blocked',
 'carrier_name',
 'location_city',
 'location_coordinates_latitude',
 'location_coordinates_longitude',
 'location_country_iso2',
 'location_country_iso3',
 'location_country_name',
 'location_county',
 'location_metro_code',
 'location_state',
 'location_time_zone_name',
 'location_time_zone_utc_offset_max',
 'location_time_zone_utc_offset_min',
 'location_zip',
 'numbering_cleansing_call_cleansed_code',
 'numbering_cleansing_call_country_code',
 'numbering_cleansing_call_max_length',
 'numbering_cleansing_call_min_length',
 'numbering_cleansing_call_phone_number',
 'numbering_cleansing_sms_cleansed_code',
 'numbering_cleansing_sms_country_code',
 'numbering_cleansing_sms_max_length',
 'numbering_cleansing_sms_min_length',
 'numbering_cleansing_sms_phone_number',
 'numbering_original_country_code',
 'numbering_original_phone_number',
 'phone_type_code',
 'phone_type_description',
 'reference_id',
 'status_code',
 'status_description',
 'status_updated_on']

MPPhoneIDa = MPPhoneID.drop(columns=droplist)

In [21]:
MPTelesignData = pd.merge(MPPhoneScores, MPPhoneIDa, how='left', left_on='numbering_original_complete_phone_number', right_on='numbering_original_complete_phone_number')

In [22]:
MPRegData = pd.read_excel('MPRegData.xlsx')

Index(['blocklisting_block_code', 'blocklisting_block_description',
       'blocklisting_blocked', 'carrier_name', 'external_id', 'location_city',
       'location_coordinates_latitude', 'location_coordinates_longitude',
       'location_country_iso2', 'location_country_iso3',
       'location_country_name', 'location_county', 'location_metro_code',
       'location_state', 'location_time_zone_name',
       'location_time_zone_utc_offset_max',
       'location_time_zone_utc_offset_min', 'location_zip',
       'numbering_cleansing_call_cleansed_code',
       'numbering_cleansing_call_country_code',
       'numbering_cleansing_call_max_length',
       'numbering_cleansing_call_min_length',
       'numbering_cleansing_call_phone_number',
       'numbering_cleansing_sms_cleansed_code',
       'numbering_cleansing_sms_country_code',
       'numbering_cleansing_sms_max_length',
       'numbering_cleansing_sms_min_length',
       'numbering_cleansing_sms_phone_number',
       'numbering_origi

In [23]:
MPRegData['numbering_original_phone_number'] = MPRegData.PhoneNumber.astype('str')

In [2]:
MPRegData[['FirstName','FirstName.1','LastName','LastName.1']]

NameError: name 'MPRegData' is not defined

In [25]:
MPRegDataCont = MPRegData[['numbering_original_phone_number','CustomerKey','FirstName','LastName']].drop_duplicates().reset_index()

In [26]:
MPVerifiData = pd.merge(MPTelesignData, MPRegDataCont, how='inner', right_on='numbering_original_phone_number', left_on='numbering_original_phone_number')

In [27]:
MPVerifiData['ContactName'] = MPVerifiData.contact_firstname + ' ' + MPVerifiData.contact_lastname
MPVerifiData['CustomerName'] = MPVerifiData.FirstName + ' ' + MPVerifiData.LastName

In [28]:
MPVerifiData['ContactNotAvail'] = np.where(MPVerifiData.ContactName.isna(),1,0)

In [29]:
MPVerifiData = MPVerifiData.drop_duplicates()

In [30]:
MPVerifiData['NameMatch'] = MPVerifiData.apply(get_ratio, axis=1)

In [4]:
MPVerifiData[MPVerifiData.ContactNotAvail == 0]['NameMatch'].hist()

NameError: name 'MPVerifiData' is not defined

In [33]:
sum(MPVerifiData[MPVerifiData.ContactNotAvail == 0]['NameMatch'] >= 50) /sum(MPVerifiData[MPVerifiData.ContactNotAvail == 0]['NameMatch'] >= 0)

0.3485178727114211